In [52]:
!pip install tensorflow==2.0.0-alpha0

In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn import model_selection, metrics
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras
import glob


import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_V2.csv', 'test_V2.csv', 'sample_submission_V2.csv']


In [54]:
def load_data(train_path='../input/train_V2.csv', test_path='../input/test_V2.csv'):
    train = pd.DataFrame.from_csv(train_path, index_col="Id")
    test = pd.DataFrame.from_csv(test_path, index_col="Id")
    
    return train, test

In [55]:
train, test = load_data()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
# train = train[:5000]
# test = test[:2000]

In [57]:
train.dropna(axis=0, inplace=True)

In [58]:
train.winPlacePerc = np.where(train.winPlacePerc >= 0.5 , 1, 0)

In [59]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train.drop('winPlacePerc', axis=1), train.winPlacePerc, test_size=0.2, shuffle=None)

In [60]:
cols = [tf.feature_column.numeric_column(i) for i in X_train.select_dtypes(exclude=['object'])]
for i in train.select_dtypes(include=['object']).columns.values:
    cols.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key=i, vocabulary_list=train[i].unique())))
# cols.append(matchType)

In [61]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function


In [62]:
def train_classification_model(
    learning_rate,
    steps,
    batch_size,
    X_train,
    y_train,
    X_test,
    y_test,
    cols,
    m_dir,
    periods):

    steps_per_period = steps / periods  

    training_input_fn = make_input_fn(X_train, y_train, shuffle=True)

    ptraining_input_fn = make_input_fn(X_train, y_train, num_epochs=1, shuffle=False)

    validation_input_fn = make_input_fn(X_test, y_test, num_epochs=1, shuffle=False)

    classifier = tf.estimator.DNNClassifier(
        feature_columns=cols,
        n_classes=2,
        hidden_units=[64, 128, 64],
        model_dir=m_dir
    )

    # Train the model, but do so inside a loop so that we can periodically assess
    # loss metrics.
    print("Training model...")
    print("LogLoss error (on validation data):")
    training_errors = []
    validation_errors = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        classifier.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        classifier.evaluate(input_fn=validation_input_fn)
        # Take a break and compute probabilities.
#         training_predictions = list(classifier.predict(input_fn=ptraining_input_fn))
#         training_probabilities = np.array([item['probabilities'] for item in training_predictions])
#         training_pred_class_id = np.array([item['class_ids'][0] for item in training_predictions])
#         training_pred_one_hot = tf.keras.utils.to_categorical(training_pred_class_id,2)

        validation_predictions = list(classifier.predict(input_fn=validation_input_fn))
#         validation_probabilities = np.array([item['probabilities'] for item in validation_predictions])    
#         validation_pred_class_id = np.array([item['class_ids'][0] for item in validation_predictions])
#         validation_pred_one_hot = tf.keras.utils.to_categorical(validation_pred_class_id,2)    
#         # Compute training and validation errors.
#         training_log_loss = metrics.log_loss(y_train, training_pred_one_hot)
#         validation_log_loss = metrics.log_loss(y_test, validation_pred_one_hot)
#         # Occasionally print the current loss.
#         print("  period %02d : %0.2f" % (period, validation_log_loss))
#         # Add the loss metrics from this period to our list.
#         training_errors.append(training_log_loss)
#         validation_errors.append(validation_log_loss)
#     print("Model training finished.")
#     # Remove event files to save disk space.
#     _ = map(os.remove, glob.glob(os.path.join(classifier.model_dir, 'events.out.tfevents*')))

#     # Calculate final predictions (not probabilities, as above).
#     final_predictions = classifier.predict(input_fn=validation_input_fn)
#     final_predictions = np.array([item['class_ids'][0] for item in final_predictions])

#     accuracy = metrics.accuracy_score(y_test, final_predictions)
#     print("Final accuracy (on validation data): %0.2f" % accuracy)

#     # Output a graph of loss metrics over periods.
#     plt.ylabel("LogLoss")
#     plt.xlabel("Periods")
#     plt.title("LogLoss vs. Periods")
#     plt.plot(training_errors, label="training")
#     plt.plot(validation_errors, label="validation")
#     plt.legend()
#     plt.show()

#     # Output a plot of the confusion matrix.
#     cm = metrics.confusion_matrix(y_test, final_predictions)
#     # Normalize the confusion matrix by row (i.e by the number of samples
#     # in each class).
#     cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
#     ax = sns.heatmap(cm_normalized, cmap="bone_r")
#     ax.set_aspect(1)
#     plt.title("Confusion matrix")
#     plt.ylabel("True label")
#     plt.xlabel("Predicted label")
#     plt.show()

    return classifier

In [63]:
model = train_classification_model(
    learning_rate=0.005,
    steps=100,
    batch_size=10,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    cols=cols,
    periods=10,
    m_dir='/tmp/model1')

Training model...
LogLoss error (on validation data):


InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

tensor_name = dnn/hiddenlayer_0/kernel; shape in shape_and_slice spec [2074748,64] does not match the shape stored in checkpoint: [9776,64]
	 [[node save/RestoreV2_1 (defined at <ipython-input-62-ebc20fb3ebec>:38) ]]

Original stack trace for 'save/RestoreV2_1':
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/sit

In [ ]:
scores1 = model.predict(input_fn=tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = test,
    y = None,
    batch_size = 1,
    num_epochs = 1,
    shuffle = False,
))

res = []
for i, p in enumerate(scores1):
    res.append(p['class_ids'][0])   

# result = pd.DataFrame(res, columns=['ImageId', 'Label'])
# result.to_csv('model1.csv', index=None)

In [ ]:
submission = pd.DataFrame(
    {"Id": test.index, "winPlacePerc": res},
    columns=["Id", "winPlacePerc"]
)
submission.to_csv("submission.csv", index=False)

In [ ]:
!cat submission.csv